creat three parsing methods that takes us from the main cusines page to a specifc cusine to a specifc recipie page
and the last one will get :
- name of the receipes
- ingridents
- url for the direction
- total time to make


-

Beautiful Soup Attempt:

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import requests

In [ ]:
url = 'https://www.allrecipes.com/cuisine-a-z-6740455'
result = requests.get(url)
doc = BeautifulSoup(result.text, "html.parser")
cuisines = doc.select('ul.loc.mntl-link-list a')

In [ ]:
cuisine_dict = {}
for link in cuisines:
    cuisine = link.get_text(strip=True)
    url = link['href']
    cuisine_dict[cuisine] = url

print(cuisine_dict)

{'Amish and Mennonite': 'https://www.allrecipes.com/recipes/732/us-recipes/amish-and-mennonite/', 'Argentinian': 'https://www.allrecipes.com/recipes/2432/world-cuisine/latin-american/south-american/argentinian/', 'Australian and New Zealander': 'https://www.allrecipes.com/recipes/228/world-cuisine/australian-and-new-zealander/', 'Austrian': 'https://www.allrecipes.com/recipes/718/world-cuisine/european/austrian/', 'Bangladeshi': 'https://www.allrecipes.com/recipes/16100/world-cuisine/asian/bangladeshi/', 'Belgian': 'https://www.allrecipes.com/recipes/719/world-cuisine/european/belgian/', 'Brazilian': 'https://www.allrecipes.com/recipes/1278/world-cuisine/latin-american/south-american/brazilian/', 'Cajun and Creole': 'https://www.allrecipes.com/recipes/272/us-recipes/cajun-and-creole/', 'Canadian': 'https://www.allrecipes.com/recipes/733/world-cuisine/canadian/', 'Chilean': 'https://www.allrecipes.com/recipes/1277/world-cuisine/latin-american/south-american/chilean/', 'Chinese': 'https:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(list(cuisine_dict.items()), columns=['Cuisine', 'URL'])
df.head()

,Cuisine,URL
0,Amish and Mennonite,https://www.allrecipes.com/recipes/732/us-reci...
1,Argentinian,https://www.allrecipes.com/recipes/2432/world-...
2,Australian and New Zealander,https://www.allrecipes.com/recipes/228/world-c...
3,Austrian,https://www.allrecipes.com/recipes/718/world-c...
4,Bangladeshi,https://www.allrecipes.com/recipes/16100/world...


In [ ]:
# Create an empty list to store recipe information
recipes_data = []

# Iterate over rows in the cuisine DataFrame
for index, row in df.iterrows():
    cuisine_url = row['URL']
    cuisine = df['Cuisine'][index]
    result = requests.get(cuisine_url)
    doc = BeautifulSoup(result.text, 'html.parser')

    # {'class': 'comp mntl-card-list-items mntl-document-card mntl-card card--image-top card card--no-image'}
    # Extract information for each recipe
    recipe_info1 = doc.find_all('a', {'class': 'comp mntl-card-list-items mntl-document-card mntl-card card card--no-image'})
    recipe_info2 = doc.find_all('a', {'class': 'comp mntl-card-list-items mntl-document-card mntl-card card--image-top card card--no-image'})
    recipe_info = recipe_info1 + recipe_info2
    for recipe_card in recipe_info:
        name = recipe_card.find('span', {'class': 'card__title-text'}).text.strip()
        url = recipe_card['href']

        # Extract information from individual recipe URLs
        if not pd.isna(url):
            result2 = requests.get(url)
            doc2 = BeautifulSoup(result2.text, 'html.parser')

            # Create a list to store ingredients
            ingredients_list = []
            ingredients_container = doc2.find('div', {'class': 'mntl-lrs-ingredients'})

            # Check if the container is found
            if ingredients_container:
                # Find the list of ingredients
                ingredients_list_element = ingredients_container.find('ul', {'class': 'mntl-structured-ingredients__list'})

                # Check if the list of ingredients is found
                if ingredients_list_element:
                    # Extract and append each ingredient to the list
                    for ingredient_item in ingredients_list_element.find_all('li', {'class': 'mntl-structured-ingredients__list-item'}):
                        ingredient = ingredient_item.find('span', {'data-ingredient-name': 'true'})
                        quantity = ingredient_item.find('span', {'data-ingredient-quantity': 'true'})
                        unit = ingredient_item.find('span', {'data-ingredient-unit': 'true'})

                        if ingredient and quantity and unit:
                            ingredient_text = f"{quantity.text.strip()} {unit.text.strip()} {ingredient.text.strip()}"
                            ingredients_list.append(ingredient_text)

            # Get total time


            # Append recipe information to the list
            recipes_data.append({
                'Name': name,
                'URL': url,
                'Cuisine': cuisine,
                'Ingredients': ingredients_list,
            })

# Create a DataFrame from the list of recipes
recipes_df = pd.DataFrame(recipes_data)

In [ ]:
recipes_df

,Name,URL,Cuisine,Ingredients
0,Best Vinegar Coleslaw,https://www.allrecipes.com/recipe/59318/amish-...,Amish and Mennonite,"[1 large head cabbage, cored and finely shredd..."
1,Pennsylvania Dutch Pickled Beets and Eggs,https://www.allrecipes.com/recipe/13743/pennsy...,Amish and Mennonite,"[8 large eggs, 2 (15 ounce) cans whole pickled..."
2,Amish Macaroni Salad,https://www.allrecipes.com/recipe/74915/amish-...,Amish and Mennonite,"[2 cups uncooked elbow macaroni, 3 large hard-..."
3,Amish Friendship Bread Starter,https://www.allrecipes.com/recipe/7063/amish-f...,Amish and Mennonite,"[1 (.25 ounce) package active dry yeast, ¼ cup..."
4,My Amish Friend's Caramel Corn,https://www.allrecipes.com/recipe/74950/my-ami...,Amish and Mennonite,"[7 quarts plain popped popcorn, 2 cups dry roa..."
...,...,...,...,...
2316,Vietnamese Grilled Pork Skewers,https://www.allrecipes.com/recipe/261122/vietn...,Vietnamese,"[1 pound pork belly, cubed, 1 fresh red chile..."
2317,Goi Ga (Vietnamese Chicken and Cabbage Salad),https://www.allrecipes.com/recipe/271155/goi-g...,Vietnamese,"[4 skinless cooked chicken breasts, shredded,..."
2318,Vietnamese Fresh Spring Rolls,https://www.allrecipes.com/recipe/24239/vietna...,Vietnamese,"[2 ounces rice vermicelli, 8 rice wrappers (8..."
2319,Pho (Vietnamese Noodle Soup),https://www.allrecipes.com/recipe/228443/authe...,Vietnamese,"[4 pounds beef soup bones (shank and knee), 1 ..."


In [ ]:
recipes_df.to_csv('recipes.csv', index=False)